In [2]:
!pip install tqdm bert-serving-client

fastai 1.0.55 requires nvidia-ml-py3, which is not installed.
tensorboard 1.14.0 has requirement protobuf>=3.6.0, but you'll have protobuf 3.5.2 which is incompatible.
tensorboard 1.14.0 has requirement setuptools>=41.0.0, but you'll have setuptools 39.1.0 which is incompatible.
thinc 6.12.1 has requirement msgpack<0.6.0,>=0.5.6, but you'll have msgpack 0.6.0 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import sagemaker
from sagemaker import get_execution_role
import json
import boto3
import numpy as np
import re
import tqdm
import pickle 
import pandas as pd

sagemaker_session = sagemaker.Session()
role = get_execution_role()
print(role) # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf
bucket = sagemaker_session.default_bucket()
print(bucket)
s3_output_location = 's3://{}/nlp/cnnoutput'.format(bucket) # This is the output location for models
prefix = 'ML_workshop' #Replace with the prefix under which you want to store the data if needed
region_name = boto3.Session().region_name

arn:aws:iam::579019700964:role/service-role/AmazonSageMaker-ExecutionRole-20190429T111678
sagemaker-us-east-1-579019700964


In [2]:
pk = open('class_mapping_r.pkl', "rb") 
index_to_label = pickle.load(pk)

s3_train_data = pd.read_csv('data/train.tsv',header=None, delimiter='\t')
s3_dev_data = pd.read_csv('data/dev.tsv',header=None, delimiter='\t')

index_to_label

{'默认分组': '__label__0',
 '结算问题（商管专用）': '__label__1',
 '无效咨询（在线专用）': '__label__2',
 '临时问题（在线专用）': '__label__3',
 '订单问题（商管专用）': '__label__4',
 '财务问题（商管专用）': '__label__5',
 '规则问题（商管专用）': '__label__6',
 '售后问题（在线专用）': '__label__7',
 '售后问题（商管专用）': '__label__8',
 'plus分销': '__label__9',
 '商品问题（商管专用）': '__label__10',
 '店铺问题（商管专用）': '__label__11',
 '售中问题（在线专用）': '__label__12',
 '售前问题（在线专用）': '__label__13',
 '社群专用': '__label__14',
 '入驻问题（商管专用）': '__label__15',
 '售后问题（蜜店宝）': '__label__16',
 '__label__0': '默认分组',
 '__label__1': '结算问题（商管专用）',
 '__label__2': '无效咨询（在线专用）',
 '__label__3': '临时问题（在线专用）',
 '__label__4': '订单问题（商管专用）',
 '__label__5': '财务问题（商管专用）',
 '__label__6': '规则问题（商管专用）',
 '__label__7': '售后问题（在线专用）',
 '__label__8': '售后问题（商管专用）',
 '__label__9': 'plus分销',
 '__label__10': '商品问题（商管专用）',
 '__label__11': '店铺问题（商管专用）',
 '__label__12': '售中问题（在线专用）',
 '__label__13': '售前问题（在线专用）',
 '__label__14': '社群专用',
 '__label__15': '入驻问题（商管专用）',
 '__label__16': '售后问题（蜜店宝）'}

In [3]:
train_raw = s3_train_data[1]
train_raw = train_raw.apply(lambda x: x if len(x)> 0 else np.nan)
train = train_raw.dropna()

test_raw = s3_dev_data[1]
test_raw = test_raw.apply(lambda x: x if len(x)> 0 else np.nan)
test = test_raw.dropna()

In [4]:
# one-hot encoding y
import sklearn.preprocessing as pre
y_raw=s3_train_data[0].astype('category')
y = y_raw[train_raw.notnull()]
# print(y_raw.head())
encoder = pre.LabelEncoder()
y = encoder.fit_transform(y)
# print(y)
with open('data/train_y.pkl', 'wb') as f:
    pickle.dump(y, f)

In [5]:
# Do the following operation in a EC2
# !pip install -U bert-serving-server bert-serving-client
# !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
# !unzip chinese_L-12_H-768_A-12.zip
# !bert-serving-start -model_dir /tmp/chinese_L-12_H-768_A-12/ -num_worker=1

  Using cached https://files.pythonhosted.org/packages/af/ef/0df9a6ce54a02d0a891d25af60e49b3a3a64d425e80c28acfee97f5ab0f2/bert_serving_server-1.9.6-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b5/2f/dd50af5b8dbde79e69f4bd2edec222eaa23d1015b03e9613411b78f9a639/bert_serving_client-1.9.6-py2.py3-none-any.whl
Requirement not upgraded as not directly required: termcolor>=1.1 in /home/ec2-user/.local/lib/python3.6/site-packages (from bert-serving-server) (1.1.0)
Requirement not upgraded as not directly required: numpy in /home/ec2-user/.local/lib/python3.6/site-packages (from bert-serving-server) (1.17.1)
  Using cached https://files.pythonhosted.org/packages/75/89/6f0ea51ffa9c2c00c0ab0460f137b16a5ab5b47e3b060c5b1fc9ca425836/pyzmq-18.1.0-cp36-cp36m-manylinux1_x86_64.whl
Requirement not upgraded as not directly required: six in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (from bert-serving-server) (1.11.0)
  Found existing installation: 

In [7]:
# Bert Preprocessing and save datasets.

from bert_serving.client import BertClient
bc = BertClient(ip='18.224.55.190') # this ip would change
train_X = []
for sentence in tqdm.tqdm(train.values):
    lst = [ x for x in re.split('。|，|！',sentence) if len(x)>2 ]
    train_X.append(bc.encode(lst))

 # write train_X
with open('data/train_X.pkl', 'wb') as f:
    pickle.dump(train_X, f)

test_X = []
for sentence in tqdm.tqdm(test.values):
    lst = [ x for x in re.split('。|，|！',sentence) if len(x)>2 ]
    test_X.append(bc.encode(lst))
with open('data/test_X.pkl', 'wb') as f:
    pickle.dump(test_X, f)
test_y = s3_train_data[0]
with open('data/test_y.pkl', 'wb') as f:
    pickle.dump(test_y, f)

    
# If the server not working:
# !wget https://sagemaker-us-east-2-069799604450.s3.us-east-2.amazonaws.com/ML_workshop/test_X.pkl
# !wget https://sagemaker-us-east-2-069799604450.s3.us-east-2.amazonaws.com/ML_workshop/test_y.pkl
# !wget https://sagemaker-us-east-2-069799604450.s3.us-east-2.amazonaws.com/ML_workshop/train_X.pkl
# !wget https://sagemaker-us-east-2-069799604450.s3.us-east-2.amazonaws.com/ML_workshop/train_y.pkl


  0%|          | 5/4250 [00:00<05:49, 12.13it/s]/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/bert_serving/client/__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)
100%|██████████| 473/473 [00:43<00:00, 10.76it/s]


In [8]:
inputs = sagemaker_session.upload_data(path='/home/ec2-user/SageMaker/gcr_sm_workshop/NLP/NLP_DL/data/', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-579019700964/ML_workshop


In [9]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(entry_point='train.py',
                    role=role,
                    framework_version='1.1.0',
                    train_instance_count=1,
                    train_instance_type='ml.c5.4xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'batchsize': 128
                    }
                    )
estimator.fit({'training': inputs})

2019-09-06 02:51:11 Starting - Starting the training job...
2019-09-06 02:51:12 Starting - Launching requested ML instances...
2019-09-06 02:52:11 Starting - Preparing the instances for training......
2019-09-06 02:53:03 Downloading - Downloading input data
2019-09-06 02:53:03 Training - Downloading the training image..
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-09-06 02:53:15,840 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-09-06 02:53:15,843 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-09-06 02:53:15,852 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-09-06 02:53:22,098 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-09-06 02:53:22,288 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2019-09-06 02:53:22,288 s

In [10]:
#check model
from sagemaker.pytorch import PyTorchModel
training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)

s3://sagemaker-us-east-1-579019700964/sagemaker-pytorch-2019-09-06-02-51-10-792/output/model.tar.gz


In [11]:
!aws s3 cp $trained_model_location .
!tar xf model.tar.gz

download: s3://sagemaker-us-east-1-579019700964/sagemaker-pytorch-2019-09-06-02-51-10-792/output/model.tar.gz to ./model.tar.gz


In [4]:
from train import *
new_model=torch.load('model.pth')
test_label = []
test_X = pickle.load(open('data/test_X.pkl', 'rb'))
for index in range(len(test_X)):
    test=torch.as_tensor(test_X[index]).unsqueeze(0)
    with torch.no_grad():
        result = new_model(test)
        label = result[0].argmax(dim=-1).numpy()[0].astype(int)
        print('label{}: '.format(index),index_to_label[encoder.inverse_transform([label])[0]])
        

usage: __main__.py [-h] [--batchsize BATCHSIZE] [--epochs EPOCHS]
__main__.py: error: unrecognized arguments: -f /home/ec2-user/.local/share/jupyter/runtime/kernel-972ee2a1-e431-48c9-88f2-c59799b4fa22.json


SystemExit: 2

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [107]:
set(test_label)

{'__label__15', '__label__9'}